In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from scipy.signal import medfilt

In [ ]:
input_filepath = './images/joker-mask.jpg'
output_filepath = './stencils/joker-mask.jpg'
num_colours = 4

In [ ]:
def make_stencil(image, num_colours):
    # Convert the image to greyscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Flatten the image into shape (-1, 1)
    flattened_image = np.reshape(gray_image, newshape=(-1, 1))
    
    # Cluster the image into num_colours distinct colours
    kmeans = KMeans(n_clusters=num_colours, random_state=0).fit_predict(flattened_image)
    
    # Recolour the image using the most different shades possible
    vfunc = np.vectorize(lambda val: val * (255 / (num_colours - 1)))
    
    # Map the recolouring function onto the image
    _image = vfunc(np.reshape(kmeans, newshape=gray_image.shape))
    
    return _image

In [ ]:
# Read in the image in grayscale
image = cv2.imread(input_filepath)


# Make it into a stencil
stencil = make_stencil(image, num_colours)


# stencil = medfilt(stencil, 3)
stencil.astype(np.uint8)

# Save the final image
cv2.imwrite(output_filepath, stencil)
